#### DETEKSI KEMATANGAN BUAH KELOMPOK 5

In [1]:
import cv2
import numpy as np
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

uploaded_image_path = None
last_result = ""

def show_histogram(image_path):
    image = cv2.imread(image_path)
    color = ('b', 'g', 'r')

    fig = Figure(figsize=(3.5, 3), dpi=100)
    plot = fig.add_subplot(111)
    for i, col in enumerate(color):
        histr = cv2.calcHist([image], [i], None, [256], [0, 256])
        plot.plot(histr, color=col)
    plot.set_title('Histogram Warna')

    for spine in plot.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(0.8)

    canvas = FigureCanvasTkAgg(fig, master=histogram_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

def detect_ripeness_orange(image_path):
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_orange = np.array([10, 100, 100])
    upper_orange = np.array([25, 255, 255])
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    orange_pixels = cv2.countNonZero(mask)
    total_pixels = image.shape[0] * image.shape[1]
    ratio = orange_pixels / total_pixels

    if ratio > 0.2:
        return "Jeruk Matang 🍊"
    else:
        return "Jeruk Belum Matang 🍋"

def upload_image():
    global uploaded_image_path
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    uploaded_image_path = file_path

    img = Image.open(file_path)
    img = img.resize((300, 300))
    img_tk = ImageTk.PhotoImage(img)
    img_label.config(image=img_tk)
    img_label.image = img_tk

    result_label.config(text="Klik 'Scan Kematangan' untuk deteksi")

    for widget in histogram_frame.winfo_children():
        widget.destroy()
    show_histogram(file_path)

def scan_image():
    global last_result
    if uploaded_image_path:
        result = detect_ripeness_orange(uploaded_image_path)
        last_result = result
        result_label.config(text=f"Hasil: {result}")
    else:
        messagebox.showinfo("Info", "Silakan upload gambar terlebih dahulu!")

def simpan_hasil():
    if last_result:
        with open("hasil_deteksi.txt", "w", encoding="utf-8") as f:
            f.write(f"Hasil Deteksi Kematangan: {last_result}\n")
        messagebox.showinfo("Berhasil", "Hasil berhasil disimpan ke hasil_deteksi.txt")
    else:
        messagebox.showwarning("Peringatan", "Belum ada hasil yang bisa disimpan!")

# GUI setup
root = Tk()
root.title("Deteksi Kematangan Buah Jeruk")
root.geometry("700x700")
root.configure(bg="#ADD8E6")

# Logo
try:
    logo_image = Image.open("logo.png")
    logo_image = logo_image.resize((100, 100))
    logo_photo = ImageTk.PhotoImage(logo_image)
    logo_label = Label(root, image=logo_photo, bg="#ADD8E6")
    logo_label.pack(pady=5)
except:
    logo_label = Label(root, text="(Logo tidak ditemukan)", bg="#ADD8E6", fg="red")
    logo_label.pack()

# Judul
title_label = Label(root, text="Deteksi Kematangan Buah Jeruk",
                    font=("Helvetica", 16, "bold"), bg="#ADD8E6", fg="darkblue")
title_label.pack(pady=5)

# Tombol Upload
upload_btn = Button(root, text="Upload Gambar Buah", command=upload_image,
                    bg="white", fg="black", font=("Arial", 12, "bold"),
                    relief="solid", bd=2)
upload_btn.pack(pady=10)

# Frame untuk gambar dan histogram disamping
img_hist_frame = Frame(root, bg="#ADD8E6")
img_hist_frame.pack(pady=10)

# Label Gambar di sebelah kiri
img_label = Label(img_hist_frame, bg="#ADD8E6")
img_label.pack(side=LEFT, padx=10)

# Frame Histogram di sebelah kanan
histogram_frame = Frame(img_hist_frame, bg="#ADD8E6")
histogram_frame.pack(side=RIGHT, padx=10)

# Tombol Scan
scan_btn = Button(root, text="Scan Kematangan", command=scan_image,
                  bg="white", fg="black", font=("Arial", 12, "bold"),
                  relief="solid", bd=2)
scan_btn.pack(pady=10)

# Frame Hasil Deteksi
result_frame = Frame(root, bg="white", bd=2, relief="solid")
result_frame.pack(pady=10)
result_label = Label(result_frame, text="", font=("Arial", 14),
                     bg="white", fg="black", padx=10, pady=5)
result_label.pack()

# Tombol Simpan
save_btn = Button(root, text="Simpan Hasil Deteksi", command=simpan_hasil,
                  bg="white", fg="black", font=("Arial", 12, "bold"),
                  relief="solid", bd=2)
save_btn.pack(pady=10)

root.mainloop()
